# Calculations of supercurrents in a proximitized semiconducting 3D nanowire

In [ ]:
# Make sure this folder is in your $PYTHONPATH

try:
    # If this can be imported, it assumes you are on the TU Delft network with access to cluster
    import hpc05
    client = hpc05.Client(profile='pbs', timeout=60)
    print("Connected to hpc05")
except ImportError:
    from ipyparallel import Client
    client = Client()
    print("Connected to local engines")

In [ ]:
dview = client[:]
dview.use_dill()
lview = client.load_balanced_view()
print(len(dview))

In [ ]:
# Standard library imports
from functools import partial
from itertools import product, repeat
import os.path
import types

# Related third party imports
import holoviews as hv
from ipyparallel import require
import kwant
import numpy as np
import pandas as pd
import sympy.interactive

# Local imports
import funcs
from funcs import constants

# next two lines because of https://github.com/ipython/ipyparallel/issues/205
%px import sys; sys.path.append('/home/bnijholt/Dropbox/Work/nanowire_current/')
%px import funcs, types, kwant

%matplotlib inline
hv.notebook_extension()
sympy.interactive.init_printing('mathjax')

# Creating some wires as example

In [ ]:
# Create system with infinite leads
syst_pars = dict(a=8, A_in_SC=True, angle=0, site_disorder=False, holes=True, L=40, L_sc=8,
                 phi=135, r1=50, r2=70, shape='circle', spin=True, with_leads=True,
                 with_shell=True, with_vlead=True)

syst, hopping = funcs.make_3d_wire(**syst_pars)
kwant.plot(syst)

In [ ]:
# Create system without leads and bigger SC regions
syst_pars['L_sc'] = 40
syst, hopping = funcs.make_3d_wire(**syst_pars)
kwant.plot(syst)

In [ ]:
# Create system without leads
syst_pars['with_leads'] = False
syst, hopping = funcs.make_3d_wire(**syst_pars)
kwant.plot(syst)

# Mean free path

In [ ]:
syst_pars = dict(a=8, A_in_SC=True, angle=0, site_disorder=True, holes=False, L_sc=8,
                 phi=135, r1=50, r2=70, shape='circle', spin=True, with_leads=True,
                 with_shell=False, with_vlead=False)

ham_pars = dict(alpha=20, B_x=0, B_y=0, B_z=0, Delta=0, g=50, mu_B=constants.mu_B,
                orbital=True, t=constants.t, V=lambda x: 0)

Ls = np.arange(80, 2000, 80)
salts = np.arange(0, 30)
disorders = np.arange(0, 160, 10)
mus = [6, 10, 15, 20, 40, 50]

vals = list(product(salts, disorders, Ls, mus))

@require('kwant', 'types', 'funcs')
def transmission(x, syst_pars=syst_pars, ham_pars=ham_pars):
    syst, hopping = funcs.make_3d_wire(**dict(**syst_pars, L=x[2]))
    p = types.SimpleNamespace(**ham_pars, salt=x[0], disorder=x[1], mu=x[3])
    smatrix = kwant.smatrix(syst, args=[p])
    return smatrix.transmission(0, 1), smatrix.num_propagating(0)

trans = lview.map_async(transmission, vals)
trans.wait_interactive()
trans = trans.result()

df = pd.DataFrame(vals, columns=['salt', 'disorder', 'L', 'mu'])
df = pd.concat((pd.DataFrame(trans, columns=['transmission', 'num_propagating']), df), axis=1)
df = df.assign(**syst_pars).assign(**ham_pars).assign(**constants.__dict__)
df = df.assign(git_hash=funcs.get_git_revision_hash())
df.to_hdf('data/mean_free_path.hdf', 'all_data', mode='w')

# Disorder

In [ ]:
syst_pars = dict(a=8, A_in_SC=True, angle=0, site_disorder=True, holes=True, phi=135,
                 r1=50, r2=70, shape='circle', spin=True, with_shell=True, with_vlead=True,
                 with_leads=True, L_sc=8, L=640)

ham_pars = dict(B_y=0, B_z=0, Delta=60, mu_B=constants.mu_B,
                t=constants.t, t_interface=7/8*constants.t, V=lambda x: 0)

T = 1

Bs = np.linspace(0, 1, 100)
orbital_bools = [False, True]
gs = [0, 50]
alphas = [0, 20]
mus = [10, 50]
disorders = [20, 50, 70]
salts = np.arange(0, 1)

vals = list(product(Bs, orbital_bools, gs, alphas, mus, disorders, salts))

@require('kwant', 'types', 'funcs')
def current(x, syst_pars=syst_pars, ham_pars=ham_pars, T=T):
    import funcs, types, kwant
    syst, hopping = funcs.make_3d_wire(**dict(**syst_pars))
    p = types.SimpleNamespace(**ham_pars, B_x=x[0], orbital=x[1], g=x[2],
                              alpha=x[3], mu=x[4], disorder=x[5], salt=x[6])
    return funcs.I_c(syst, hopping, p, T=x[0])

for i, chunk in enumerate(funcs.chunks(vals, 5000)):
    fname = 'tmp/I_c_B_x_disorder_single_{}.hdf'.format(i)
    if not os.path.exists(fname):
        I = lview.map_async(current, chunk)
        I.wait_interactive()
        current_phase = I.result()
        
        df = pd.DataFrame(chunk, columns=['B_x', 'orbital', 'g', 'alpha', 'mu', 'disorder', 'salts'])
        df = pd.concat((pd.DataFrame(current_phase), df), axis=1)
        df = df.assign(**syst_pars).assign(**ham_pars).assign(**constants.__dict__)
        df = df.assign(git_hash=funcs.get_git_revision_hash())
        df.to_hdf(fname, 'all_data', mode='w')


# No disorder $B_x$ vs $I_c$:

In [ ]:
syst_pars = dict(a=8, A_in_SC=True, angle=0, site_disorder=False, holes=True, phi=135,
                 r1=50, r2=70, shape='circle', spin=True, 
                 with_shell=True, with_vlead=True)

ham_pars = dict(B_y=0, B_z=0, Delta=60, mu_B=constants.mu_B,
                t=constants.t, t_interface=7/8*constants.t, V=lambda x: 0)

Ts = [0.1, 0.5, 1]
orbital_bools = [False, True]
gs = [0, 50]
alphas = [0, 20]
mus = [10, 20, 30, 40, 50, 60]
Ls = [80, 160, 320, 640]
leads = [(True, 8), (False, 400)]
Bs = np.linspace(0, 1, 100)

vals = list(product(Ts, Bs, orbital_bools, gs, alphas, mus, Ls, leads))

@require('kwant', 'types', 'funcs')
def current(x, syst_pars=syst_pars, ham_pars=ham_pars):
    syst, hopping = funcs.make_3d_wire(**dict(**syst_pars, L=x[6], with_leads=x[7][0], L_sc=x[7][1]))
    p = types.SimpleNamespace(**ham_pars, B_x=x[1], orbital=x[2], g=x[3], alpha=x[4], mu=x[5])
    return funcs.I_c(syst, hopping, p, T=x[0])

for i, chunk in enumerate(funcs.chunks(vals, 50000)):
    fname = 'tmp/I_c_B_x_no_disorder{}.hdf'.format(i)
    if not os.path.exists(fname):
        I = lview.map_async(current, chunk)
        I.wait_interactive()
        current_phase = I.result()
        
        df = pd.DataFrame(chunk, columns=['T', 'B_x', 'orbital', 'g', 'alpha', 'mu', 'L', 'leads'])
        df[['with_leads', 'L_sc']] = df.pop('leads').apply(pd.Series)
        df = pd.concat((pd.DataFrame(current_phase), df), axis=1)
        df = df.assign(**syst_pars).assign(**ham_pars).assign(**constants.__dict__)
        df = df.assign(git_hash=funcs.get_git_revision_hash())
        df.to_hdf(fname, 'all_data', mode='w')

# $I_c(B_x, \mu)$

In [ ]:
syst_pars = dict(a=8, A_in_SC=True, angle=0, site_disorder=False, holes=True, L=640,
                 L_sc=8, phi=135, r1=50, r2=70, shape='circle',
                 spin=True, with_leads=True, with_shell=True, with_vlead=True)

ham_pars = dict(B_y=0, B_z=0, Delta=60, mu_B=constants.mu_B,
                t=constants.t, t_interface=7/8*constants.t, V=lambda x: 0)

T = 1

orbital_bools = [False, True]
gs = [0, 50]
alphas = [0, 20]
mus = np.linspace(0, 50, 100)
Bs = np.linspace(0, 1, 100)

vals = list(product(Bs, orbital_bools, gs, alphas, mus))

@require('kwant', 'types', 'funcs')
def fun(x, syst_pars=syst_pars, ham_pars=ham_pars, T=T):
    syst, hopping = funcs.make_3d_wire(**syst_pars)
    p = types.SimpleNamespace(**ham_pars, B_x=x[0], orbital=x[1], g=x[2], alpha=x[3], mu=x[4])
    return funcs.I_c(syst, hopping, p, T=T)

for i, chunk in enumerate(funcs.chunks(vals, 50000)):
    fname = 'tmp/I_c_B_x_mu_2D_plots_{}.hdf'.format(i)
    if not os.path.exists(fname):
        I = lview.map_async(fun, chunk)

        I.wait_interactive()

        current_phase = I.result()

        df = pd.DataFrame(chunk, columns=['B_x', 'orbital', 'g', 'alpha', 'mu'])
        df = pd.concat((pd.DataFrame(current_phase), df), axis=1)
        df = df.assign(**syst_pars).assign(**ham_pars).assign(**constants.__dict__)
        df = df.assign(T=T, git_hash=funcs.get_git_revision_hash())
        df.to_hdf(fname, 'all_data', mode='w')

# Field in y

In [ ]:
syst_pars = dict(a=8, A_in_SC=True, angle=0, site_disorder=False, holes=True, L=640,
                 L_sc=8, phi=135, r1=50, r2=70, shape='circle', spin=True,
                 with_leads=True, with_shell=True, with_vlead=True)

ham_pars = dict(B_x=0, B_z=0, Delta=60, mu_B=constants.mu_B,
                t=constants.t, t_interface=7/8*constants.t, V=lambda x: 0)
T = 1

orbital_bools = [False, True]
gs = [0, 50]
alphas = [0, 20]
mus = [10, 50]
Bs = np.linspace(0, 1, 100)

vals = list(product(Bs, orbital_bools, gs, alphas, mus))[:100]

@require('kwant', 'types', 'funcs')
def fun(x, syst_pars=syst_pars, ham_pars=ham_pars, T=T):
    syst, hopping = funcs.make_3d_wire(**syst_pars)
    p = types.SimpleNamespace(**ham_pars, B_y=x[0], orbital=x[1], g=x[2], alpha=x[3], mu=x[4])
    return funcs.I_c(syst, hopping, p, T=T)

current_phase = lview.map_async(fun, vals)
current_phase.wait_interactive()
current_phase = current_phase.result()

df = pd.DataFrame(vals, columns=['B_x', 'orbital', 'g', 'alpha', 'mu'])
df = pd.concat((pd.DataFrame(current_phase), df), axis=1)
df = df.assign(**syst_pars).assign(**ham_pars).assign(**constants.__dict__)
df = df.assign(T=T).assign(git_hash=funcs.get_git_revision_hash())

# Rotation of the field

![](https://upload.wikimedia.org/wikipedia/commons/thumb/4/4f/3D_Spherical.svg/558px-3D_Spherical.svg.png)

In [ ]:
syst_pars = dict(a=8, A_in_SC=True, angle=0, site_disorder=False, holes=True, L=640, L_sc=8,
                 phi=135, r1=50, r2=70, shape='circle', spin=True, 
                 with_leads=True, with_shell=True, with_vlead=True)

ham_pars = dict(alpha=20, B_y=0, Delta=60, g=50, 
                mu=50, mu_B=constants.mu_B, orbital=True, 
                t=constants.t, t_interface=7/8*constants.t, V=lambda x: 0)

T = 1

Bs = np.linspace(0, 1, 100)
angles = np.arange(0, 360, 15)
B_xs = [B * np.cos(angle * np.pi / 180) for angle in angles for B in Bs]
B_zs = [B * np.sin(angle * np.pi / 180) for angle in angles for B in Bs]
vals = list(zip(B_xs, B_zs))

@require('kwant', 'types', 'funcs')
def fun(x, syst_pars=syst_pars, ham_pars=ham_pars, T=T):
    p = types.SimpleNamespace(**ham_pars, B_x=x[0], B_z=x[1])
    syst, hopping = funcs.make_3d_wire(**syst_pars)
    return funcs.I_c(syst, hopping, p, T)

I = lview.map_async(fun, vals)
I.wait_interactive()
current_phase = I.result()

df = pd.DataFrame(vals, columns=['B_x', 'B_z'])
df = pd.concat((pd.DataFrame(current_phase), df), axis=1)
df = df.assign(**syst_pars).assign(**ham_pars).assign(**constants.__dict__)
df = df.assign(T=T).assign(git_hash=funcs.get_git_revision_hash())
df.to_hdf('rotation_of_field.hdf', 'all_data', mode='w')

# Induced gap

In [ ]:
# Create system with infinite leads
syst_pars = dict(a=8, A_in_SC=True, angle=0, site_disorder=False, holes=True,
                 L=80, L_sc=8, phi=135, r1=50, r2=70, shape='circle',
                 spin=True, with_leads=True, with_shell=True, with_vlead=True)

ham_pars = dict(alpha=20, B_x=0, B_y=0, B_z=0, g=50, 
                mu_B=constants.mu_B, orbital=True, 
                t=constants.t, t_interface=7/8*constants.t, V=lambda x: 0)

mus = np.linspace(2, 30, 50)
Deltas = np.linspace(0, 100, 50)

@require('kwant', 'types', 'funcs', 'numpy')
def evs(x, syst_pars=syst_pars, ham_pars=ham_pars):
    import numpy as np
    syst, _ = funcs.make_3d_wire(**syst_pars)
    lead = syst.leads[1]
    p = types.SimpleNamespace(**ham_pars, Delta=x[0], mu=x[1])
    h0 = lead.cell_hamiltonian(args=[p])
    t0 = lead.inter_cell_hopping(args=[p])
    ham = h0 + t0 + t0.conj().T
    ev = np.linalg.eigvalsh(ham)
    return np.abs(ev).min()

vals = list(product(Deltas, mus))
gap = lview.map_async(evs, vals)
gap.wait_interactive()
gap = np.reshape(gap.result(), (len(Deltas), -1)) 
max_gap = max(np.min(gap, axis=1))
print("Max gap is {} meV".format(max_gap))

In [ ]:
hv.Curve((Deltas, gap.min(axis=1)), kdims=['$\Delta$'], vdims=['$E_{gap}$'], label="$E_{gap}$ over range of $\Delta$'s")

In [ ]:
Delta_ind = {delta: hv.Path((mus, evs), kdims=['$\mu$', ('E_gap', '$E_{gap}$')]) 
      for evs, delta in zip(gap, Deltas)}

hm = hv.HoloMap(Delta_ind, kdims=['$\Delta$'])
(hm.select(E_gap=(0.2, 0.40)) * hv.HLine(0.25))

# Gate

In [ ]:
# Plot test
syst_pars = dict(a=8, A_in_SC=True, angle=0, site_disorder=False, holes=True,
                   L=640, L_sc=8, phi=135, r1=50, r2=70, shape='circle',
                   spin=True, with_leads=True, with_shell=True, with_vlead=True)

syst, hopping = funcs.make_3d_wire(**syst_pars)
gate_fun = funcs.gate(syst, 100, 160)
sites = [gate_fun(pos[0]) for pos in [i.pos for i in syst.sites]]
kwant.plot(syst, site_lw=0, site_color=sites, colorbar=False)

In [ ]:
syst_pars = dict(a=8, A_in_SC=True, angle=0, site_disorder=False, holes=True,
                 L=640, L_sc=8, phi=135, r1=50, r2=70, shape='circle',
                 spin=True, with_leads=True, with_shell=True, with_vlead=True)

ham_pars = dict(alpha=20, B_y=0, B_z=0, Delta=60, g=50, mu=50,
                mu_B=constants.mu_B, orbital=True, t=constants.t,
                t_interface=7/8*constants.t)

T = 1
Bs = np.linspace(0, 1, 100)
Vs = np.linspace(0, 40, 100)

gate_sizes = [80, 160, 320, 640]

@require('kwant', 'types', 'funcs')
def fun(x, syst_pars=syst_pars, ham_pars=ham_pars, T=T):
    syst, hopping = funcs.make_3d_wire(**syst_pars)
    p = types.SimpleNamespace(**ham_pars, B_x=x[2], V=funcs.gate(syst, x[1], x[0]))
    return funcs.I_c(syst, hopping, p, T)

vals = list(product(gate_sizes, Vs, Bs))

for i, chunk in enumerate(funcs.chunks(vals, 50000)):
    fname = "current_as_function_of_gate_and_B_x__gate_size_{}_nm.h5".format(i)
    if not os.path.exists(fname):
        I = lview.map_async(fun, chunk)
        I.wait_interactive()
        current_phase = I.result()

        df = pd.DataFrame(chunk, columns=['gate_size', 'V', 'B_x'])
        df = pd.concat((pd.DataFrame(current_phase), df), axis=1)
        df = df.assign(**syst_pars).assign(**ham_pars).assign(**constants.__dict__)
        df = df.assign(T=T).assign(git_hash=funcs.get_git_revision_hash())
        df.to_hdf(fname, 'all_data', mode='w')

# gate with disorder

In [ ]:
syst_pars = dict(a=8, A_in_SC=True, angle=0, site_disorder=True, holes=True,
                 L=640, L_sc=8, phi=135, r1=50, r2=70, shape='circle',
                 spin=True, with_leads=True, with_shell=True, with_vlead=True)

ham_pars = dict(alpha=20, B_y=0, B_z=0, Delta=60, g=50, mu=50,
                mu_B=constants.mu_B, orbital=True, t=constants.t,
                t_interface=7/8*constants.t, salt=0)

T = 1
Bs = np.linspace(0, 1, 100)
Vs = np.linspace(0, 40, 100)
gate_sizes = [80, 160, 320, 640]
disorders = [20, 50, 70]

@require('kwant', 'types', 'funcs')
def fun(x, syst_pars=syst_pars, ham_pars=ham_pars, T=T):
    syst, hopping = funcs.make_3d_wire(**syst_pars)
    p = types.SimpleNamespace(**ham_pars, B_x=x[3],
                              V=funcs.gate(syst, V=x[2], gate_size=x[0]),
                              disorder=x[1])
    return funcs.I_c(syst, hopping, p, T)

vals = list(product(gate_sizes, disorders, Vs, Bs))

for i, chunk in enumerate(funcs.chunks(vals, 5000)):
    fname = "tmp/current_as_function_of_gate_and_B_x__gate_size_{}_nm_disorder.h5".format(i)
    if not os.path.exists(fname):
        I = lview.map_async(fun, chunk)
        I.wait_interactive()
        current_phase = I.result()

        df = pd.DataFrame(chunk, columns=['gate_size', 'disorder', 'V', 'B_x'])
        df = pd.concat((pd.DataFrame(current_phase), df), axis=1)
        df = df.assign(**syst_pars).assign(**ham_pars).assign(**constants.__dict__)
        df = df.assign(T=T).assign(git_hash=funcs.get_git_revision_hash())
        df.to_hdf(fname, 'all_data', mode='w')